# NYC COVID-19 Cases - Data Collection, Exploring, Prepping, and Cleaning

## Purpose: 

#### Collect local NYC COVID-19 data (this notebook) and and plot by coordinates on a map (separate notebook)

## Resources & References:

#### NYC Health COVID-19 Data - https://github.com/nychealth/coronavirus-data/blob/master/tests-by-zcta.csv

#### ZCTA Code to Polygons Source - https://mydata.iowa.gov/Boundaries/ZIP-Code-Tabulation-Areas-ZCTAs-/4nkg-rrrc

#### Zip Codes & FIPS (not used) - https://www.kaggle.com/danofer/zipcodes-county-fips-crosswalk/version/1#

#### Geojson with FIPS (not used) - https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json

# Import Packages

In [2]:
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
import math
import geopandas

# Set local working directory

In [3]:
my_path = '/Users/Dave/Desktop/CODE/DBProjects/DBP0001_Local-COVID-Tracker/GitHub/'

# Read in Data from "NYC Health" & Inspect

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/tests-by-zcta.csv')

print(df.head())

print('\n')

print(df.describe())

print('\n')

print(df.info())

print('\n')

print(df.dtypes)

print('\n Total Positive Cases to Date: ' + str(df['Positive'].sum()))

   MODZCTA  Positive  Total  zcta_cum.perc_pos
0      NaN      1844   2051              89.91
1  10001.0       254    563              45.12
2  10002.0       695   1325              52.45
3  10003.0       342    812              42.12
4  10004.0        24     62              38.71


            MODZCTA     Positive        Total  zcta_cum.perc_pos
count    177.000000   178.000000   178.000000         178.000000
mean   10810.378531   723.466292  1286.707865          53.587472
std      578.173317   505.132059   804.608775           9.531953
min    10001.000000    14.000000    54.000000          25.930000
25%    10301.000000   348.750000   648.750000          46.807500
50%    11109.000000   589.500000  1101.500000          55.535000
75%    11361.000000  1042.500000  1891.500000          60.697500
max    11697.000000  2749.000000  4079.000000          89.910000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 4 columns):
 #   Column             N

# Send Zip Codes to List, Convert to int (and remove NaN)

In [5]:
ziplist=df['MODZCTA'].tolist()
ziplistfix=[]

for i in ziplist:
    if math.isnan(i):
        continue
    else:
        ziplistfix.append(int(i))
        
print(ziplistfix)

print('\nTotal # of Zip Codes in NYC Health Dataframe: ' + str(len(ziplistfix)))

[10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10010, 10011, 10012, 10013, 10014, 10016, 10017, 10018, 10019, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10044, 10065, 10069, 10075, 10128, 10280, 10282, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10312, 10314, 10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465, 10466, 10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475, 11004, 11101, 11102, 11103, 11104, 11105, 11106, 11109, 11201, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210, 11211, 11212, 11213, 11214, 11215, 11216, 11217, 11218, 11219, 11220, 11221, 11222, 11223, 11224, 11225, 11226, 11228, 11229, 11230, 11231, 11232, 11233, 11234, 11235, 11236, 11237, 11238, 11239, 11354, 11355, 11356, 11357, 11358, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11372

# Collect Polygon/Geo Coordinate Data for ZCTA Codes from "Iowa.Gov"

### Download CSV via: https://mydata.iowa.gov/Boundaries/ZIP-Code-Tabulation-Areas-ZCTAs-/4nkg-rrrc
### Read into Dataframe and inspect

In [6]:
df2=pd.read_csv(my_path+'ZIP_Code_Tabulation_Areas__ZCTAs_.csv')

In [92]:
###IF IN ZIP FORMAT

# import zipfile
# zf = zipfile.ZipFile(my_path+'ZIP_Code_Tabulation_Areas__ZCTAs_.csv.zip') 
# df2 = pd.read_csv(zf.open('ZIP_Code_Tabulation_Areas__ZCTAs_.csv'))

In [7]:
print(df2.head())

print('\n')

print(df2.describe())

print('\n')

print(df2.info)

print('\n')

print(df2.dtypes)

   ZCTA Code                                           the_geom  \
0      76565  MULTIPOLYGON (((-98.416145 31.663624, -98.4042...   
1      32334  MULTIPOLYGON (((-84.884013 30.373846999999994,...   
2      88310  MULTIPOLYGON (((-106.195761 32.746862, -106.17...   
3      81040  MULTIPOLYGON (((-105.501286 37.598603, -105.49...   
4      56560  MULTIPOLYGON (((-96.839548 47.006727999999995,...   

                                        polygon_geom  \
0  POLYGON ((-98.416145 31.663624, -98.40428 31.6...   
1  POLYGON ((-84.884013 30.373846999999994, -84.8...   
2  POLYGON ((-106.195761 32.746862, -106.176111 3...   
3  POLYGON ((-105.501286 37.598603, -105.493273 3...   
4  POLYGON ((-96.839548 47.006727999999995, -96.8...   

                                        centroid  
0  POINT (-98.33188436718751 31.647865195312495)  
1  POINT (-84.78215753320312 30.378027208984367)  
2   POINT (-106.00594445800783 32.7774532685547)  
3  POINT (-105.16623194042968 37.81005166894531)  
4  PO

# Create new Dataframe where df2 is filtered to match Zip Codes in df

### ...and rename "ZCTA Code" and "the_geom" columns

In [8]:
df_geomatch=df2[df2['ZCTA Code'].isin(ziplistfix)]

df_geomatch=df_geomatch.rename(columns={'the_geom': 'geometry'})

df_geomatch=df_geomatch.rename(columns={'ZCTA Code': 'ZCTA_Code'})

print(df_geomatch.head())

print('\n')

print(df_geomatch.describe())

print('\n')

print(df_geomatch.info)

df_geomatch_i = df_geomatch.index

df_geomatch_i_rowcount = len(df_geomatch_i)

print('\nNumber of rows in df_geomatch: ' + str(df_geomatch_i_rowcount))

      ZCTA_Code                                           geometry  \
380       10280  MULTIPOLYGON (((-74.01858899999999 40.712201, ...   
989       11358  MULTIPOLYGON (((-73.807514 40.762178999999996,...   
1879      11225  MULTIPOLYGON (((-73.96289399999999 40.663798, ...   
1937      11221  MULTIPOLYGON (((-73.948185 40.690242, -73.9453...   
1967      11411  MULTIPOLYGON (((-73.74864300000002 40.70158699...   

                                           polygon_geom  \
380   POLYGON ((-74.01858899999999 40.712201, -74.01...   
989   POLYGON ((-73.807514 40.762178999999996, -73.8...   
1879  POLYGON ((-73.96289399999999 40.663798, -73.96...   
1937  POLYGON ((-73.948185 40.690242, -73.945349 40....   
1967  POLYGON ((-73.74864300000002 40.70158699999999...   

                                           centroid  
380    POINT (-74.01690924352604 40.71025141632766)  
989       POINT (-73.79669671875001 40.76080428125)  
1879  POINT (-73.95431249265435 40.663038155147255)  
1937   P

# Send df2 geometry/multipolygon series and polygon_geom/polygon series to separate lists to inspect full entries

### ...and in case further manipulation is needed

In [9]:
multipoly_lst = pd.Series(df_geomatch.geometry.values,index=df_geomatch['ZCTA_Code']).to_dict()

poly_lst = pd.Series(df_geomatch.polygon_geom.values,index=df_geomatch['ZCTA_Code']).to_dict()

example_1 = multipoly_lst[10280]

example_2 = poly_lst[10280]

print(example_1)

print('\n')

print(example_2)

MULTIPOLYGON (((-74.01858899999999 40.712201, -74.016508 40.713224, -74.01736799999999 40.713378, -74.016198 40.714766999999995, -74.01359699999999 40.713623, -74.014697 40.709779, -74.015956 40.70668, -74.016131 40.706244, -74.017575 40.70684, -74.018008 40.705664999999996, -74.016628 40.704799, -74.015678 40.70125600000001, -74.0142519151327 40.7003146857495, -74.0168 40.701794, -74.0194854329352 40.70690774995101, -74.01858899999999 40.712201)))


POLYGON ((-74.01858899999999 40.712201, -74.016508 40.713224, -74.01736799999999 40.713378, -74.016198 40.714766999999995, -74.01359699999999 40.713623, -74.014697 40.709779, -74.015956 40.70668, -74.016131 40.706244, -74.017575 40.70684, -74.018008 40.705664999999996, -74.016628 40.704799, -74.015678 40.70125600000001, -74.0142519151327 40.7003146857495, -74.0168 40.701794, -74.0194854329352 40.70690774995101, -74.01858899999999 40.712201))


# Send df_geomatch Dataframe to csv

In [10]:
df_geomatch.to_csv(my_path+'df_geomatch.csv')

# Import and use wkt and json module to create a geojson column in df_geomatch Dataframe and Inspect

In [11]:
from geomet import wkt
import json

#Apply function to create new column
df_geomatch["geojson"] = df_geomatch["polygon_geom"].apply(lambda x: json.dumps(wkt.loads(x)))

print(df_geomatch.head())

print('\n')

print(df_geomatch.dtypes)

      ZCTA_Code                                           geometry  \
380       10280  MULTIPOLYGON (((-74.01858899999999 40.712201, ...   
989       11358  MULTIPOLYGON (((-73.807514 40.762178999999996,...   
1879      11225  MULTIPOLYGON (((-73.96289399999999 40.663798, ...   
1937      11221  MULTIPOLYGON (((-73.948185 40.690242, -73.9453...   
1967      11411  MULTIPOLYGON (((-73.74864300000002 40.70158699...   

                                           polygon_geom  \
380   POLYGON ((-74.01858899999999 40.712201, -74.01...   
989   POLYGON ((-73.807514 40.762178999999996, -73.8...   
1879  POLYGON ((-73.96289399999999 40.663798, -73.96...   
1937  POLYGON ((-73.948185 40.690242, -73.945349 40....   
1967  POLYGON ((-73.74864300000002 40.70158699999999...   

                                           centroid  \
380    POINT (-74.01690924352604 40.71025141632766)   
989       POINT (-73.79669671875001 40.76080428125)   
1879  POINT (-73.95431249265435 40.663038155147255)   
1937

# Inspect a single df_geomatch geojson entry

In [12]:
df_geomatch.iloc[0,4]

'{"type": "Polygon", "coordinates": [[[-74.01858899999999, 40.712201], [-74.016508, 40.713224], [-74.01736799999999, 40.713378], [-74.016198, 40.714766999999995], [-74.01359699999999, 40.713623], [-74.014697, 40.709779], [-74.015956, 40.70668], [-74.016131, 40.706244], [-74.017575, 40.70684], [-74.018008, 40.705664999999996], [-74.016628, 40.704799], [-74.015678, 40.70125600000001], [-74.0142519151327, 40.7003146857495], [-74.0168, 40.701794], [-74.0194854329352, 40.70690774995101], [-74.01858899999999, 40.712201]]]}'

# Prep Dataframes for Merge

### Drop NaN in original df, cast MODZCTA to int, make a copy of df (df_prefinal) and rename MODZCTA to ZCTA_Code... and make sure row count of df_prefinal matches length of df_geomatch

In [13]:
df.dropna(inplace=True)

df['MODZCTA']=df['MODZCTA'].astype(int)

df_prefinal = df

df_prefinal = df_prefinal.rename(columns={'MODZCTA': 'ZCTA_Code'})

df_prefinal_i = df_prefinal.index

df_prefinal_i_rowcount = len(df_prefinal_i)

print('\nNumber of rows in df_prefinal: ' + str(df_prefinal_i_rowcount))

if df_geomatch_i_rowcount == df_prefinal_i_rowcount:
    print('\nROW COUNTS MATCH!')
else:
    print('\nDouble check data. Row counts don\'t match')


Number of rows in df_prefinal: 177

ROW COUNTS MATCH!


# Merge Dataframes df_geomatch and df_prefinal on ZCTA_Code & Inspect

In [14]:
df_final_merged = df_geomatch.merge(df_prefinal, how = 'inner', on = ['ZCTA_Code'])

print(df_final_merged['Total'].max())

print('\n')

print(df_final_merged.head())

print('\n')

print(df_final_merged.info)

print('\n')

print(df_final_merged.describe())

print('\n')

print(df_final_merged.dtypes)

4079


   ZCTA_Code                                           geometry  \
0      10280  MULTIPOLYGON (((-74.01858899999999 40.712201, ...   
1      11358  MULTIPOLYGON (((-73.807514 40.762178999999996,...   
2      11225  MULTIPOLYGON (((-73.96289399999999 40.663798, ...   
3      11221  MULTIPOLYGON (((-73.948185 40.690242, -73.9453...   
4      11411  MULTIPOLYGON (((-73.74864300000002 40.70158699...   

                                        polygon_geom  \
0  POLYGON ((-74.01858899999999 40.712201, -74.01...   
1  POLYGON ((-73.807514 40.762178999999996, -73.8...   
2  POLYGON ((-73.96289399999999 40.663798, -73.96...   
3  POLYGON ((-73.948185 40.690242, -73.945349 40....   
4  POLYGON ((-73.74864300000002 40.70158699999999...   

                                        centroid  \
0   POINT (-74.01690924352604 40.71025141632766)   
1      POINT (-73.79669671875001 40.76080428125)   
2  POINT (-73.95431249265435 40.663038155147255)   
3   POINT (-73.92605128125001 40.693249031249

# Cast ZCTA_Code column to str

In [15]:
df_final_merged['ZCTA_Code']=df_final_merged['ZCTA_Code'].astype(str)

# Send df_final_merged to csv

In [16]:
df_final_merged.to_csv(my_path+'NYC-covid-map-dataframe.csv')

# Create a geojson file from df_final_merged with geo/coordinate data and COVID data

In [19]:
def df_to_geojson(df, properties, zcta='ZCTA_Code',gj='geojson'):
    geojson = {'type':'FeatureCollection','features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':'',
                  'id':[]}
        feature['geometry']= json.loads(row[gj])
        feature['id']= row[zcta]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols=["Positive","Total",'ZCTA_Code']
geojson_dict = df_to_geojson(df_final_merged,properties=cols)
geojson_str = json.dumps(geojson_dict, indent=2)

output_filename = my_path+'NYC-covid-map-dataset.json'
with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))

# Read in geojson to inspect

In [18]:
with open(my_path+'NYC-covid-map-dataset.json') as file:
    ny_geojson = json.load(file)
    print(json.dumps(ny_geojson, indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "feature",
      "properties": {
        "Positive": 32,
        "Total": 100,
        "ZCTA_Code": "10280"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -74.01858899999999,
              40.712201
            ],
            [
              -74.016508,
              40.713224
            ],
            [
              -74.01736799999999,
              40.713378
            ],
            [
              -74.016198,
              40.714766999999995
            ],
            [
              -74.01359699999999,
              40.713623
            ],
            [
              -74.014697,
              40.709779
            ],
            [
              -74.015956,
              40.70668
            ],
            [
              -74.016131,
              40.706244
            ],
            [
              -74.017575,
              40.70684
   